In [2]:
import requests
 
class census_api:
    baseurl = "https://api.census.gov/data/2019/cps/school/oct?get=HEFAMINC,PELASTGD"
    
    def __init__(self):
        #self.paramx = {'dataset'}
        self.resp = requests.get(self.baseurl).json()
         
    def output(self):
        return self.resp
census_header = census_api().output()

In [9]:
import censusdata, pandas as pd

def import_censusdata(year, variables):
    census_data = censusdata.download('acs5', year, 
                                censusdata.censusgeo([('state','*'), ('county', '*')]), variables)

    dictionary = {'B02001_001E':'Total Pop Estimate', 'B01001_006E':'Males 15-17',\
                  'B01001A_006E':'White Males 15-17','B01001B_006E':'Black Males 15-17', 'B01001I_006E':'Hispanic Males 15-17',\
                   'B19001_001E':'HHI', 'B19001_016E':'HHI 150K-200K','B19001A_001E':'HHI 220K+'}
    census_data.rename(columns =dictionary, inplace=True)
    
    
    for index, data in census_data.iterrows():
        string_county = index.name.split(',')
        census_data.loc[index,'County'], census_data.loc[index,'State']= string_county[0], string_county[1]

       # print(string_county)
    census_data.reset_index(drop=True,inplace=True)
    census_dataframe = pd.DataFrame(census_data)

    return census_dataframe.groupby(['State', 'County']).agg({'Total Pop Estimate':'sum',
                                                                'Males 15-17':'sum', 'Black Males 15-17':'sum', 'Hispanic Males 15-17':'sum',\
                                                            'HHI 220K+':'sum'})

males_census_2019df = import_censusdata(2019, ['B02001_001E', 'B01001_006E','B01001A_006E','B01001_006E' , 'B01001B_006E', 'B01001I_006E',\
                                                'B19001_001E', 'B19001_016E','B19001A_001E'])
#males_census_2019df.sort_values(by = 'Males 15-17', ascending=False)


In [11]:
import censusdata, pandas as pd

def import_censusdata0(year, variables, geo_agg):
    census_data = censusdata.download('acs5', year, 
                                censusdata.censusgeo([('state','*'), geo_agg]), variables)

    dictionary = {'B02001_001E': 'Total Pop Estimate '+str(year), 'B19001_001E':'HHI', 'B19001_016E':'HHI 150K-200K','B19001A_001E':'HHI 220K+',\
    'B01001_006E':'Males 15-17', 'B01001A_006E':'White Males 15-17','B01001B_006E':'Black Males 15-17', 'B01001I_006E':'Hispanic Males 15-17',\
    'B01001_030E': 'Females 15-17', 'B01001A_021E':'White Females 15-17',' B01001B_021E':'Black Females 15-17','B01001I_021E': 'Hispanic Females 15-17'}
    
    census_data.rename(columns =dictionary, inplace=True)
    for index, data in census_data.iterrows():
        census_data.loc[index,'County'], census_data.loc[index,'State']= index.name, index.params()[0][1]

    #census_data.reset_index(drop=True,inplace=True)
    census_dataframe = pd.DataFrame(census_data)

    return census_dataframe#.groupby(['State', 'County']).agg({'Total Pop Estimate'+str(year):'sum',
                                                           #     'Males 15-17':'sum', 'Black Males 15-17':'sum', 'Hispanic Males 15-17':'sum',\
                                                           # 'HHI 220K+':'sum'})

male_vars = ['B02001_001E', 'B01001_006E','B01001A_006E','B01001_006E' , 'B01001B_006E', 'B01001I_006E',\
                                                'B19001_001E', 'B19001_016E','B19001A_001E']

female_vars = ['B02001_001E', 'B01001_030E', 'B01001A_021E','B01001B_021E','B01001I_021E']

males_census_2019df = import_censusdata0(2019, male_vars, ('county', '*') )
males_census_2018df = import_censusdata0(2018, male_vars, ('county', '*') )

In [12]:
growth_trend = pd.concat([males_census_2019df['Total Pop Estimate 2019'], males_census_2018df['Total Pop Estimate 2018']], axis=1)
growth_trend['% delta'] = growth_trend['Total Pop Estimate 2019'] / growth_trend['Total Pop Estimate 2018']


In [7]:
#this variant uses groupings for the variables
import requests
 
class census_api:
    baseurl = "https://api.census.gov/data/2019/acs/acs5?get=NAME,group(B02001)&for=zip%20code%20tabulation%20area:*&in=state:*"
    
    def __init__(self):
        #self.paramx = {'dataset'}
        self.resp = requests.get(self.baseurl).json()
         
    def output(self):
        return self.resp
census_header = census_api().output()

In [14]:
def school_data(year, dataset, var, grade):
    import urllib, json, pandas as pd
    #dir(urllib3.request.urlencode)
    if dataset == 'ccd':
         url = "https://educationdata.urban.org/api/v1/schools/"+dataset+"/"+var+"/"+str(year)+"/"
         
    elif dataset == 'crdc':
         url = "https://educationdata.urban.org/api/v1/schools/"+dataset+"/"+var+"/"+str(year)+"/"+grade+"/race/sex/"
    print(url)
    response = urllib.request.urlopen(url)
    retention = json.loads(response.read())
    return pd.json_normalize(retention, record_path= ['results'])

retention2017 = school_data(2017, 'ccd','directory', False)


https://educationdata.urban.org/api/v1/schools/ccd/directory/2017/


In [ ]:
data = pd.json_normalize(data, record_path = ['results'])
data1 = data.sort_values(by=['students_retained'], ascending=False)

In [ ]:
retention2019

,year,ncessch,school_id,school_name,leaid,lea_name,state_leaid,seasch,street_mailing,city_mailing,...,high_cedp,middle_cedp,ungrade_cedp,enrollment,state_leg_district_lower,state_leg_district_upper,ncessch_num,congress_district_id,direct_certification,lunch_program
0,2017,010000500870,00870,Albertville Middle School,0100005,Albertville City,AL-101,101-0010,600 E Alabama Ave,Albertville,...,0,1,0,807.0,01026,01009,10000500870,104,290.0,2.0
1,2017,010000500871,00871,Albertville High School,0100005,Albertville City,AL-101,101-0020,402 E McCord Ave,Albertville,...,1,0,0,1436.0,01026,01009,10000500871,104,432.0,2.0
2,2017,010000500879,00879,Evans Elementary School,0100005,Albertville City,AL-101,101-0110,901 W McKinney Ave,Albertville,...,0,0,0,850.0,01026,01009,10000500879,104,349.0,2.0
3,2017,010000500889,00889,Albertville Elementary School,0100005,Albertville City,AL-101,101-0200,1100 Horton Rd,Albertville,...,0,0,0,906.0,01026,01009,10000500889,104,388.0,2.0
4,2017,010000501616,01616,Big Spring Lake Kindergarten School,0100005,Albertville City,AL-101,101-0035,257 Country Club Rd,Albertville,...,0,0,0,418.0,01026,01009,10000501616,104,174.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2017,040150001053,01053,Desert Valley School,0401500,Bullhead City School District,AZ-4378,4378-5584,1066 Marina Blvd.,Bullhead City,...,0,0,0,454.0,04005,04005,40150001053,404,NaN,2.0
2996,2017,040150001528,01528,Diamondback Elementary School,0401500,Bullhead City School District,AZ-4378,4378-5585,2550 Tesota Way,Bullhead City,...,0,0,0,555.0,04005,04005,40150001528,404,NaN,2.0
2997,2017,040150001757,01757,Fox Creek Jr High School,0401500,Bullhead City School District,AZ-4378,4378-6048,3101 Desert Sky Boulevard,Bullhead City,...,0,1,0,486.0,04005,04005,40150001757,404,NaN,2.0
2998,2017,040150002184,02184,Sunrise Elementary,0401500,Bullhead City School District,AZ-4378,4378-79655,2645 West Landon Drive,Bullhead City,...,0,0,0,356.0,04005,04005,40150002184,404,NaN,2.0
